### https://www.kaggle.com/vincentman0403/dcgan-on-mnist 


In [1]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
import time
import os
import csv
from PIL import Image

Using TensorFlow backend.


In [2]:
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)
latent_dim = 100
root = 'DCGAN_results/'

# plotting data structure
train_hist = {}
train_hist['D_losses'] = []
train_hist['G_losses'] = []
train_hist['per_epoch_ptimes'] = []
train_hist['total_ptime'] = []
train_hist['accuracy'] = []
train_hist['Model'] = []


# optimizer = Adam(0.0002, 0.5)

In [3]:
def build_generator(pick_model='Deep_BN'):
    if (pick_model == 'Deep_BN') | (pick_model == 'Deeper_G_BN') | (pick_model == 'Super_Deeper_G_BN'):
        model = Sequential()
        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))
        model.summary()
    
    elif (pick_model == 'Shallow_BN') | (pick_model == 'Deeper_D_BN'): 
        model = Sequential()
        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Activation("relu"))
        model.add(Conv2D(channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))
        model.summary()

    elif (pick_model == 'Super_Shallow_BN') | (pick_model == 'Super_Deeper_D_BN'): 
        model = Sequential()
        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
#         model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Activation("relu"))
        model.add(Conv2D(channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))
        model.summary()

    elif pick_model == 'Shallow_Drop':
        model = Sequential()
        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(Dropout(0.25))
        model.add(UpSampling2D())
        model.add(Activation("relu"))
        model.add(Conv2D(channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))
        model.summary()
    
    elif pick_model == 'Drop':
        model = Sequential()
        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(Dropout(0.25))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(Dropout(0.25))
        model.add(Activation("relu"))
        model.add(Conv2D(channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))
        model.summary()
        
        
        
    noise = Input(shape=(latent_dim,))
    img = model(noise)
    
    return Model(noise, img)

## Define a function to build a discriminator

In [4]:
# ['Super_Shallow_BN', 'Super_Deeper_G_BN', 'Super_Deeper_G_BN' ]

def build_discriminator(pick_model='Deep_BN'):
    if (pick_model == 'Deep_BN') | (pick_model == 'Deeper_D_BN') | (pick_model == 'Super_Deeper_D_BN'):
        model = Sequential()
        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        
    elif (pick_model == 'Shallow_BN') | (pick_model == 'Deeper_G_BN'): 
        model = Sequential()
        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        
    elif (pick_model == 'Super_Shallow_BN') | (pick_model == 'Super_Deeper_G_BN'): 
        model = Sequential()
        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
        model.add(BatchNormalization(momentum=0.8))
#         model.add(LeakyReLU(alpha=0.2))
#         model.add(Dropout(0.25))
#         model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
#         model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

    elif pick_model == 'Shallow_Drop':
        model = Sequential()
        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
        model.add(Dropout(0.25))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(Dropout(0.25))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        
    elif pick_model == 'Drop':
        model = Sequential()
        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
        model.add(Dropout(0.25))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(Dropout(0.25))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(Dropout(0.25))        
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
    
    
    img = Input(shape=img_shape)
    validity = model(img)
    return Model(img, validity)

## Define a function to train GAN

In [5]:
def train(epochs, batch_size=128, save_interval=50, ratio = (1,1), pick_model = 'Deep_BN', learning_rate = 0.0002): ## ratio G:D
    os.makedirs('images', exist_ok=True)
    
    # Load the dataset
    (X_train, _), (_, _) = mnist.load_data()

    # Rescale -1 to 1
    X_train = X_train / 127.5 - 1.
    X_train = np.expand_dims(X_train, axis=3)

    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    
    for epoch in range(epochs):
        # Select a random real images
        epoch_start_time = time.time()
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_imgs = X_train[idx]

        # Sample noise and generate a batch of fake images
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        fake_imgs = generator.predict(noise)
          
        #implement ratio of D  
        if epoch % ratio[0] == 0:
            # Train the discriminator
            D_loss_real = discriminator.train_on_batch(real_imgs, valid)
            D_loss_fake = discriminator.train_on_batch(fake_imgs, fake)
            D_loss = 0.5 * np.add(D_loss_real, D_loss_fake)
        #implement ratio of G
        if epoch % ratio[1] == 0:
            # Train the generator
            g_loss = combined.train_on_batch(noise, valid)
            
        # get epoch timing
        epoch_end_time = time.time()
        per_epoch_ptime = epoch_end_time - epoch_start_time
            
        # save data for plotting and csv
        train_hist['D_losses'].append(D_loss[0])
        train_hist['G_losses'].append(g_loss)
        train_hist['per_epoch_ptimes'].append(per_epoch_ptime)    
        train_hist['accuracy'].append(100 * D_loss[1])
        train_hist['Model'].append(pick_model)
        
        # If at save interval
        if epoch % save_interval == 0:
            # Print the progress
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f] [epoch time: %.2f]" % (epoch, D_loss[0], 100 * D_loss[1], g_loss, per_epoch_ptime))
            # Save generated image samples
            save_imgs(epoch, pick_model, ratio, learning_rate)

In [6]:
def save_imgs(epoch, pick_model, ratio, learning_rate):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            cnt += 1
    os.makedirs(root + 'images/'+  pick_model + '/'+ str(ratio[0]) + '_' + str(ratio[1]) + '/' , exist_ok=True)
    fig.savefig(root + 'images/'+  pick_model + '/'+ str(ratio[0]) + '_' + str(ratio[1]) + '/' + str(epoch) + '_' + str(learning_rate) + '.png')
    plt.close()

In [7]:
def show_train_hist(hist, show = False, save = False, path = 'Train_hist.png'):
    x = range(len(hist['D_losses']))
    
    y1 = hist['D_losses']
    y2 = hist['G_losses']

    plt.plot(x, y1, label='D_loss')
    plt.plot(x, y2, label='G_loss')

    plt.xlabel('Epoch')
    plt.ylabel('Loss')

    plt.legend(loc=4)
    plt.grid(True)
    plt.tight_layout()
    
    
    if save:
        
        plt.savefig(path)

    if show:
        plt.show()
    else:
        plt.close()

## Train GAN

In [8]:
ratio_array = [ (1,1) , (1,2), (1,3), (3,1), (2,1), (10,1), (1,10)]
# ratio_array = [ (1,30), (1,60), (1,90), (30,1), (60,1), (90,1) ] # rip 

model_array =  ['Deep_BN', 'Super_Shallow_BN', 'Super_Deeper_G_BN', 'Super_Deeper_D_BN' ]# ['Deep_BN', 'Drop', 'Shallow_BN', 'Shallow_Drop', 'Deeper_G_BN', 'Deeper_D_BN']

optimizer_array = [0.0002] #[0.000025, 0.0000125] #[0.0001, 0.00005, 0.000025, 0.0000125] # 2, 4, 8, 16 time smaller
for pick_model in model_array:
    
    
    for learning_rate in optimizer_array:
    
        
        #----------------------------------------LOOP OVER RATIOS--------------------------------#
        for ratio in ratio_array:
            #---------------------------COMPILE SELECTED MODELS--------------------------------------#
            # build discriminator
            optimizer = Adam(learning_rate, 0.5)

            discriminator = build_discriminator(pick_model=pick_model)
            discriminator.compile(loss='binary_crossentropy',
                                  optimizer=optimizer,
                                  metrics=['accuracy'])

            # build generator
            generator = build_generator(pick_model=pick_model)
            z = Input(shape=(100,))
            img = generator(z)

            # For the combined model we will only train the generator
            discriminator.trainable = False

            # The discriminator takes generated images as input and determines validity
            valid = discriminator(img)

            # The combined model  (stacked generator and discriminator)
            # Trains the generator to fool the discriminator
            combined = Model(z, valid)
            combined.compile(loss='binary_crossentropy', optimizer=optimizer)

            epochs = int(20000*(learning_rate/0.0002))
            #----------------------------------------EXECUTION-----------------------------------#
            start = time.time()

            train(epochs=epochs+1, batch_size=64, save_interval=epochs//100, ratio=ratio, pick_model=pick_model, learning_rate=learning_rate) ## ratio G:D

            end = time.time()

            #--------------------------------------GET--INFO-------------------------------------#
            elapsed_train_time = 'elapsed training time: {} min, {} sec '.format(int((end - start) / 60),
                                                                                 int((end - start) % 60))
            train_hist['total_ptime'].append(elapsed_train_time)

            print(elapsed_train_time)
            os.makedirs(root + 'hist/', exist_ok=True)  
            show_train_hist(train_hist, save=True, path=root + 'hist/' + str(ratio[0]) + '_' + str(ratio[1]) + pick_model+ '_' + str(learning_rate) + '.png')
            # save hist data to csv
            os.makedirs(root + 'hist_csv/', exist_ok=True) 
            with open(root+ 'hist_csv/' + str(ratio[0]) + '_' + str(ratio[1]) + pick_model + '_' +str(learning_rate) + '.csv', 'w') as f:
                for key in train_hist.keys():
                    f.write("%s,%s\n"%(key,train_hist[key]))

            # save weights


            # remove old data structure
            del train_hist
            # redefine
            train_hist = {}
            train_hist['D_losses'] = []
            train_hist['G_losses'] = []
            train_hist['per_epoch_ptimes'] = []
            train_hist['total_ptime'] = []
            train_hist['accuracy'] = []
            train_hist['Model'] = []

    

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_1

C:\Users\Jack\venv\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 1.008466, acc.: 41.41%] [G loss: 0.685648] [epoch time: 4.38]
200 [D loss: 0.786295, acc.: 51.56%] [G loss: 0.952255] [epoch time: 0.08]
400 [D loss: 0.737603, acc.: 50.00%] [G loss: 1.080907] [epoch time: 0.08]
600 [D loss: 0.722964, acc.: 53.91%] [G loss: 0.964403] [epoch time: 0.08]
800 [D loss: 0.755879, acc.: 50.00%] [G loss: 0.975229] [epoch time: 0.08]
1000 [D loss: 0.662793, acc.: 58.59%] [G loss: 1.205528] [epoch time: 0.08]
1200 [D loss: 0.726496, acc.: 56.25%] [G loss: 0.919586] [epoch time: 0.08]
1400 [D loss: 0.683884, acc.: 53.91%] [G loss: 1.001845] [epoch time: 0.08]
1600 [D loss: 0.707490, acc.: 54.69%] [G loss: 0.989343] [epoch time: 0.08]
1800 [D loss: 0.699541, acc.: 57.03%] [G loss: 0.980808] [epoch time: 0.08]
2000 [D loss: 0.672161, acc.: 59.38%] [G loss: 1.020246] [epoch time: 0.08]
2200 [D loss: 0.648324, acc.: 64.06%] [G loss: 0.914875] [epoch time: 0.08]
2400 [D loss: 0.683703, acc.: 55.47%] [G loss: 0.980410] [epoch time: 0.08]
2600 [D loss: 0.658

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 6272)              633472    
_________________________________________________________________
reshape_2 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
up_sampling2d_3 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 14, 14, 128)       147584    
_________________________________________________________________
batch_normalization_9 (Batch (None, 14, 14, 128)       512       
_________________________________________________________________
activation_4 (Activation)    (None, 14, 14, 128)       0         
_________________________________________________________________
up_sampling2d_4 (UpSampling2 (None, 28, 28, 128)       0         
__________

16600 [D loss: 0.443559, acc.: 78.12%] [G loss: 3.102589] [epoch time: 0.08]
16800 [D loss: 0.434371, acc.: 82.81%] [G loss: 1.837837] [epoch time: 0.08]
17000 [D loss: 0.257286, acc.: 92.19%] [G loss: 2.639184] [epoch time: 0.08]
17200 [D loss: 0.725681, acc.: 57.81%] [G loss: 0.886708] [epoch time: 0.08]
17400 [D loss: 0.417361, acc.: 84.38%] [G loss: 1.607180] [epoch time: 0.08]
17600 [D loss: 0.414896, acc.: 82.03%] [G loss: 1.606326] [epoch time: 0.08]
17800 [D loss: 0.294145, acc.: 91.41%] [G loss: 1.958528] [epoch time: 0.08]
18000 [D loss: 0.514707, acc.: 74.22%] [G loss: 1.336584] [epoch time: 0.08]
18200 [D loss: 0.377778, acc.: 86.72%] [G loss: 2.229562] [epoch time: 0.08]
18400 [D loss: 0.574477, acc.: 70.31%] [G loss: 2.381286] [epoch time: 0.08]
18600 [D loss: 0.422223, acc.: 83.59%] [G loss: 1.029370] [epoch time: 0.08]
18800 [D loss: 0.316643, acc.: 88.28%] [G loss: 1.888835] [epoch time: 0.08]
19000 [D loss: 0.428740, acc.: 78.91%] [G loss: 1.714338] [epoch time: 0.08]

...

17400 [D loss: 0.000000, acc.: 100.00%] [G loss: 15.772021] [epoch time: 0.06]
17600 [D loss: 0.000000, acc.: 100.00%] [G loss: 15.823763] [epoch time: 0.03]
17800 [D loss: 0.000001, acc.: 100.00%] [G loss: 15.858974] [epoch time: 0.03]
18000 [D loss: 0.000000, acc.: 100.00%] [G loss: 15.986110] [epoch time: 0.06]
18200 [D loss: 0.000001, acc.: 100.00%] [G loss: 16.018991] [epoch time: 0.03]
18400 [D loss: 0.000000, acc.: 100.00%] [G loss: 15.804789] [epoch time: 0.03]
18600 [D loss: 0.000000, acc.: 100.00%] [G loss: 15.933472] [epoch time: 0.06]
18800 [D loss: 0.000001, acc.: 100.00%] [G loss: 15.782086] [epoch time: 0.03]
19000 [D loss: 0.000001, acc.: 100.00%] [G loss: 16.021976] [epoch time: 0.03]
19200 [D loss: 0.000000, acc.: 100.00%] [G loss: 15.852834] [epoch time: 0.06]
19400 [D loss: 0.000000, acc.: 100.00%] [G loss: 15.985603] [epoch time: 0.03]
19600 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.065451] [epoch time: 0.03]
19800 [D loss: 0.000000, acc.: 100.00%] [G loss: 15.

7600 [D loss: 0.000004, acc.: 100.00%] [G loss: 14.773712] [epoch time: 0.06]
7800 [D loss: 0.000006, acc.: 100.00%] [G loss: 15.229453] [epoch time: 0.06]
8000 [D loss: 0.000004, acc.: 100.00%] [G loss: 15.430396] [epoch time: 0.06]
8200 [D loss: 0.000002, acc.: 100.00%] [G loss: 15.427793] [epoch time: 0.06]
8400 [D loss: 0.000002, acc.: 100.00%] [G loss: 15.338893] [epoch time: 0.06]
8600 [D loss: 0.000001, acc.: 100.00%] [G loss: 15.315569] [epoch time: 0.06]
8800 [D loss: 0.000005, acc.: 100.00%] [G loss: 15.495934] [epoch time: 0.06]
9000 [D loss: 0.000002, acc.: 100.00%] [G loss: 15.404234] [epoch time: 0.06]
9200 [D loss: 0.000001, acc.: 100.00%] [G loss: 15.175163] [epoch time: 0.06]
9400 [D loss: 0.000002, acc.: 100.00%] [G loss: 15.366219] [epoch time: 0.06]
9600 [D loss: 0.000002, acc.: 100.00%] [G loss: 15.212543] [epoch time: 0.06]
9800 [D loss: 0.000002, acc.: 100.00%] [G loss: 15.687287] [epoch time: 0.06]
10000 [D loss: 0.000001, acc.: 100.00%] [G loss: 15.881649] [epo

0 [D loss: 1.101225, acc.: 28.91%] [G loss: 0.693757] [epoch time: 19.50]
200 [D loss: 1.072402, acc.: 32.81%] [G loss: 1.435952] [epoch time: 0.06]
400 [D loss: 0.421081, acc.: 81.25%] [G loss: 2.483145] [epoch time: 0.06]
600 [D loss: 0.161892, acc.: 92.97%] [G loss: 3.862711] [epoch time: 0.06]
800 [D loss: 0.030424, acc.: 100.00%] [G loss: 4.925918] [epoch time: 0.06]
1000 [D loss: 0.017954, acc.: 100.00%] [G loss: 6.024903] [epoch time: 0.06]
1200 [D loss: 0.012253, acc.: 100.00%] [G loss: 6.469545] [epoch time: 0.06]
1400 [D loss: 0.006189, acc.: 100.00%] [G loss: 7.363674] [epoch time: 0.06]
1600 [D loss: 0.004999, acc.: 100.00%] [G loss: 6.928174] [epoch time: 0.06]
1800 [D loss: 0.002927, acc.: 100.00%] [G loss: 7.448262] [epoch time: 0.06]
2000 [D loss: 0.002898, acc.: 100.00%] [G loss: 7.456986] [epoch time: 0.06]
2200 [D loss: 0.002222, acc.: 100.00%] [G loss: 7.384526] [epoch time: 0.06]
2400 [D loss: 0.002412, acc.: 100.00%] [G loss: 8.140359] [epoch time: 0.06]
2600 [D l

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_56 (Dense)             (None, 6272)              633472    
_________________________________________________________________
reshape_28 (Reshape)         (None, 7, 7, 128)         0         
_________________________________________________________________
up_sampling2d_55 (UpSampling (None, 14, 14, 128)       0         
_________________________________________________________________
batch_normalization_98 (Batc (None, 14, 14, 128)       512       
_________________________________________________________________
up_sampling2d_56 (UpSampling (None, 28, 28, 128)       0         
_________________________________________________________________
activation_69 (Activation)   (None, 28, 28, 128)       0         
_________________________________________________________________
conv2d_140 (Conv2D)          (None, 28, 28, 1)         1153      
__________

17400 [D loss: 0.000001, acc.: 100.00%] [G loss: 16.069040] [epoch time: 0.06]
17600 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.089207] [epoch time: 0.06]
17800 [D loss: 0.000009, acc.: 100.00%] [G loss: 16.004593] [epoch time: 0.06]
18000 [D loss: 0.000002, acc.: 100.00%] [G loss: 16.096933] [epoch time: 0.06]
18200 [D loss: 0.000001, acc.: 100.00%] [G loss: 16.115349] [epoch time: 0.06]
18400 [D loss: 0.000001, acc.: 100.00%] [G loss: 16.111862] [epoch time: 0.06]
18600 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.083355] [epoch time: 0.06]
18800 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.067066] [epoch time: 0.06]
19000 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095] [epoch time: 0.06]
19200 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095] [epoch time: 0.06]
19400 [D loss: 0.000001, acc.: 100.00%] [G loss: 16.118095] [epoch time: 0.06]
19600 [D loss: 0.000001, acc.: 100.00%] [G loss: 16.118095] [epoch time: 0.06]
19800 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.

In [9]:
ratio_array = [ (1,30), (1,60), (1,90), (30,1), (60,1), (90,1) ] # rip 

model_array =  ['Deep_BN'] #, 'Super_Shallow_BN', 'Super_Deeper_G_BN', 'Super_Deeper_D_BN' ]# ['Deep_BN', 'Drop', 'Shallow_BN', 'Shallow_Drop', 'Deeper_G_BN', 'Deeper_D_BN']

optimizer_array = [0.0002] #[0.000025, 0.0000125] #[0.0001, 0.00005, 0.000025, 0.0000125] # 2, 4, 8, 16 time smaller
for pick_model in model_array:
    
    
    for learning_rate in optimizer_array:
    
        
        #----------------------------------------LOOP OVER RATIOS--------------------------------#
        for ratio in ratio_array:
            #---------------------------COMPILE SELECTED MODELS--------------------------------------#
            # build discriminator
            optimizer = Adam(learning_rate, 0.5)

            discriminator = build_discriminator(pick_model=pick_model)
            discriminator.compile(loss='binary_crossentropy',
                                  optimizer=optimizer,
                                  metrics=['accuracy'])

            # build generator
            generator = build_generator(pick_model=pick_model)
            z = Input(shape=(100,))
            img = generator(z)

            # For the combined model we will only train the generator
            discriminator.trainable = False

            # The discriminator takes generated images as input and determines validity
            valid = discriminator(img)

            # The combined model  (stacked generator and discriminator)
            # Trains the generator to fool the discriminator
            combined = Model(z, valid)
            combined.compile(loss='binary_crossentropy', optimizer=optimizer)

            epochs = int(20000*(learning_rate/0.0002))
            #----------------------------------------EXECUTION-----------------------------------#
            start = time.time()

            train(epochs=epochs+1, batch_size=64, save_interval=epochs//100, ratio=ratio, pick_model=pick_model, learning_rate=learning_rate) ## ratio G:D

            end = time.time()

            #--------------------------------------GET--INFO-------------------------------------#
            elapsed_train_time = 'elapsed training time: {} min, {} sec '.format(int((end - start) / 60),
                                                                                 int((end - start) % 60))
            train_hist['total_ptime'].append(elapsed_train_time)

            print(elapsed_train_time)
            os.makedirs(root + 'hist/', exist_ok=True)  
            show_train_hist(train_hist, save=True, path=root + 'hist/' + str(ratio[0]) + '_' + str(ratio[1]) + pick_model+ '_' + str(learning_rate) + '.png')
            # save hist data to csv
            os.makedirs(root + 'hist_csv/', exist_ok=True) 
            with open(root+ 'hist_csv/' + str(ratio[0]) + '_' + str(ratio[1]) + pick_model + '_' +str(learning_rate) + '.csv', 'w') as f:
                for key in train_hist.keys():
                    f.write("%s,%s\n"%(key,train_hist[key]))

            # save weights


            # remove old data structure
            del train_hist
            # redefine
            train_hist = {}
            train_hist['D_losses'] = []
            train_hist['G_losses'] = []
            train_hist['per_epoch_ptimes'] = []
            train_hist['total_ptime'] = []
            train_hist['accuracy'] = []
            train_hist['Model'] = []

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_141 (Conv2D)          (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_85 (LeakyReLU)   (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_85 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_142 (Conv2D)          (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_29 (ZeroPaddi (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_99 (Batc (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_86 (LeakyReLU)   (None, 8, 8, 64)          0         
__________

9400 [D loss: 0.020397, acc.: 100.00%] [G loss: 3.681775] [epoch time: 0.05]
9600 [D loss: 0.010948, acc.: 100.00%] [G loss: 5.321174] [epoch time: 0.09]
9800 [D loss: 0.009540, acc.: 100.00%] [G loss: 7.284524] [epoch time: 0.05]
10000 [D loss: 0.003292, acc.: 100.00%] [G loss: 6.708129] [epoch time: 0.05]
10200 [D loss: 0.015250, acc.: 100.00%] [G loss: 6.014953] [epoch time: 0.08]
10400 [D loss: 0.055338, acc.: 98.44%] [G loss: 7.568379] [epoch time: 0.05]
10600 [D loss: 0.034366, acc.: 99.22%] [G loss: 10.883080] [epoch time: 0.05]
10800 [D loss: 0.012835, acc.: 100.00%] [G loss: 7.588521] [epoch time: 0.08]
11000 [D loss: 0.010750, acc.: 100.00%] [G loss: 6.469763] [epoch time: 0.05]
11200 [D loss: 0.014628, acc.: 100.00%] [G loss: 5.876101] [epoch time: 0.05]
11400 [D loss: 0.016321, acc.: 100.00%] [G loss: 5.782694] [epoch time: 0.08]
11600 [D loss: 0.096904, acc.: 97.66%] [G loss: 4.899748] [epoch time: 0.05]
11800 [D loss: 0.014140, acc.: 100.00%] [G loss: 3.813360] [epoch tim

0 [D loss: 0.939692, acc.: 41.41%] [G loss: 0.697980] [epoch time: 25.56]
200 [D loss: 0.001298, acc.: 100.00%] [G loss: 12.621004] [epoch time: 0.05]
400 [D loss: 0.000086, acc.: 100.00%] [G loss: 0.192312] [epoch time: 0.05]
600 [D loss: 0.000226, acc.: 100.00%] [G loss: 0.845407] [epoch time: 0.09]
800 [D loss: 0.220818, acc.: 94.53%] [G loss: 0.542989] [epoch time: 0.05]
1000 [D loss: 0.128902, acc.: 95.31%] [G loss: 4.182220] [epoch time: 0.05]
1200 [D loss: 0.006895, acc.: 100.00%] [G loss: 0.085682] [epoch time: 0.09]
1400 [D loss: 0.003847, acc.: 100.00%] [G loss: 0.009816] [epoch time: 0.05]
1600 [D loss: 0.000947, acc.: 100.00%] [G loss: 2.586396] [epoch time: 0.05]
1800 [D loss: 0.002316, acc.: 100.00%] [G loss: 2.190244] [epoch time: 0.09]
2000 [D loss: 0.041863, acc.: 99.22%] [G loss: 0.195136] [epoch time: 0.05]
2200 [D loss: 0.009103, acc.: 100.00%] [G loss: 4.863677] [epoch time: 0.05]
2400 [D loss: 0.056977, acc.: 98.44%] [G loss: 4.725718] [epoch time: 0.09]
2600 [D l

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_62 (Dense)             (None, 6272)              633472    
_________________________________________________________________
reshape_31 (Reshape)         (None, 7, 7, 128)         0         
_________________________________________________________________
up_sampling2d_61 (UpSampling (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_159 (Conv2D)          (None, 14, 14, 128)       147584    
_________________________________________________________________
batch_normalization_112 (Bat (None, 14, 14, 128)       512       
_________________________________________________________________
activation_77 (Activation)   (None, 14, 14, 128)       0         
_________________________________________________________________
up_sampling2d_62 (UpSampling (None, 28, 28, 128)       0         
__________

16400 [D loss: 0.001746, acc.: 100.00%] [G loss: 8.209297] [epoch time: 0.05]
16600 [D loss: 0.000588, acc.: 100.00%] [G loss: 7.885602] [epoch time: 0.05]
16800 [D loss: 0.004622, acc.: 100.00%] [G loss: 0.251873] [epoch time: 0.05]
17000 [D loss: 0.002333, acc.: 100.00%] [G loss: 8.844582] [epoch time: 0.05]
17200 [D loss: 0.000886, acc.: 100.00%] [G loss: 8.393699] [epoch time: 0.05]
17400 [D loss: 0.000639, acc.: 100.00%] [G loss: 8.836848] [epoch time: 0.05]
17600 [D loss: 0.000846, acc.: 100.00%] [G loss: 9.284134] [epoch time: 0.05]
17800 [D loss: 0.003277, acc.: 100.00%] [G loss: 5.246515] [epoch time: 0.05]
18000 [D loss: 0.001233, acc.: 100.00%] [G loss: 7.146752] [epoch time: 0.09]
18200 [D loss: 0.034862, acc.: 98.44%] [G loss: 3.711475] [epoch time: 0.05]
18400 [D loss: 0.003569, acc.: 100.00%] [G loss: 7.367589] [epoch time: 0.05]
18600 [D loss: 0.001599, acc.: 100.00%] [G loss: 7.083160] [epoch time: 0.05]
18800 [D loss: 0.003965, acc.: 100.00%] [G loss: 8.609360] [epoch

5400 [D loss: 0.963460, acc.: 21.09%] [G loss: 0.948679] [epoch time: 0.10]
5600 [D loss: 0.901194, acc.: 24.22%] [G loss: 0.798109] [epoch time: 0.06]
5800 [D loss: 0.867516, acc.: 24.22%] [G loss: 0.921384] [epoch time: 0.06]
6000 [D loss: 0.852182, acc.: 33.59%] [G loss: 1.048419] [epoch time: 0.10]
6200 [D loss: 0.881878, acc.: 33.59%] [G loss: 0.814583] [epoch time: 0.06]
6400 [D loss: 0.957296, acc.: 17.19%] [G loss: 0.770979] [epoch time: 0.06]
6600 [D loss: 0.902366, acc.: 20.31%] [G loss: 0.937082] [epoch time: 0.10]
6800 [D loss: 0.867394, acc.: 27.34%] [G loss: 0.846662] [epoch time: 0.06]
7000 [D loss: 0.919398, acc.: 23.44%] [G loss: 0.745283] [epoch time: 0.06]
7200 [D loss: 0.842730, acc.: 32.03%] [G loss: 0.941629] [epoch time: 0.10]
7400 [D loss: 0.834944, acc.: 34.38%] [G loss: 0.714629] [epoch time: 0.06]
7600 [D loss: 0.822537, acc.: 31.25%] [G loss: 0.813719] [epoch time: 0.06]
7800 [D loss: 0.757298, acc.: 40.62%] [G loss: 1.019201] [epoch time: 0.10]
8000 [D loss

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_66 (Dense)             (None, 6272)              633472    
_________________________________________________________________
reshape_33 (Reshape)         (None, 7, 7, 128)         0         
_________________________________________________________________
up_sampling2d_65 (UpSampling (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_173 (Conv2D)          (None, 14, 14, 128)       147584    
_________________________________________________________________
batch_normalization_122 (Bat (None, 14, 14, 128)       512       
_________________________________________________________________
activation_83 (Activation)   (None, 14, 14, 128)       0         
_________________________________________________________________
up_sampling2d_66 (UpSampling (None, 28, 28, 128)       0         
__________

16600 [D loss: 0.780733, acc.: 40.62%] [G loss: 0.769145] [epoch time: 0.06]
16800 [D loss: 0.851618, acc.: 26.56%] [G loss: 0.819372] [epoch time: 0.10]
17000 [D loss: 0.789271, acc.: 36.72%] [G loss: 0.741055] [epoch time: 0.06]
17200 [D loss: 0.818159, acc.: 32.81%] [G loss: 0.795352] [epoch time: 0.06]
17400 [D loss: 0.823231, acc.: 30.47%] [G loss: 0.777661] [epoch time: 0.10]
17600 [D loss: 0.807343, acc.: 31.25%] [G loss: 0.745355] [epoch time: 0.06]
17800 [D loss: 0.864043, acc.: 25.78%] [G loss: 0.737910] [epoch time: 0.06]
18000 [D loss: 0.832558, acc.: 36.72%] [G loss: 0.758253] [epoch time: 0.10]
18200 [D loss: 0.762025, acc.: 37.50%] [G loss: 0.767543] [epoch time: 0.06]
18400 [D loss: 0.789542, acc.: 38.28%] [G loss: 0.755037] [epoch time: 0.06]
18600 [D loss: 0.810547, acc.: 31.25%] [G loss: 0.728491] [epoch time: 0.10]
18800 [D loss: 0.787322, acc.: 36.72%] [G loss: 0.732548] [epoch time: 0.07]
19000 [D loss: 0.784629, acc.: 38.28%] [G loss: 0.775444] [epoch time: 0.06]

5800 [D loss: 1.077234, acc.: 7.81%] [G loss: 0.834507] [epoch time: 0.06]
6000 [D loss: 1.060455, acc.: 8.59%] [G loss: 0.781269] [epoch time: 0.06]
6200 [D loss: 1.032753, acc.: 14.06%] [G loss: 0.757090] [epoch time: 0.07]
6400 [D loss: 1.031021, acc.: 19.53%] [G loss: 0.851789] [epoch time: 0.06]
6600 [D loss: 1.040898, acc.: 17.19%] [G loss: 0.776707] [epoch time: 0.06]
6800 [D loss: 1.008062, acc.: 15.62%] [G loss: 0.835400] [epoch time: 0.07]
7000 [D loss: 0.981231, acc.: 24.22%] [G loss: 0.828314] [epoch time: 0.06]
7200 [D loss: 0.933887, acc.: 20.31%] [G loss: 0.943349] [epoch time: 0.10]
7400 [D loss: 0.933380, acc.: 22.66%] [G loss: 0.940605] [epoch time: 0.06]
7600 [D loss: 1.101680, acc.: 11.72%] [G loss: 0.824743] [epoch time: 0.07]
7800 [D loss: 1.037287, acc.: 14.84%] [G loss: 0.662348] [epoch time: 0.06]
8000 [D loss: 1.027871, acc.: 8.59%] [G loss: 0.673166] [epoch time: 0.07]
8200 [D loss: 0.946934, acc.: 24.22%] [G loss: 0.901264] [epoch time: 0.07]
8400 [D loss: 0

In [10]:
# os.makedirs('saved_model_weights', exist_ok=True)
# generator.save_weights('saved_model_weights/generator_weights.h5')
# discriminator.save_weights('saved_model_weights/discriminator_weights.h5')
# combined.save_weights('saved_model_weights/combined_weights.h5')

## Show generated MNIST images